<a href="https://colab.research.google.com/github/EleonoraBartolomucci/Fairness/blob/master/Faceplusplus/LindoMichoacan_ElaboraFotoRandom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install clarifai

     |████████████████████████████████| 133kB 3.4MB/s 
  Created wheel for clarifai: filename=clarifai-2.6.2-cp36-none-any.whl size=188403 sha256=e2421f89107fc91b33ba24a47874ff54436e3b89de54e145642d45cde43738d6
  Stored in directory: /root/.cache/pip/wheels/17/2c/3b/d89ffd4572633f70872165cedf61813ac8047461ed5e1e3768
Successfully built clarifai


In [0]:
#Da cambiare:
#nome del csv e cartella da cui prendo le foto (foto ordinate come "ultima modifica e freccetta in giù")

import json
import csv
from pydrive.auth import GoogleAuth 
from pydrive.drive import GoogleDrive

# apro e parso il file .json
def convert(response,file):

  with open('utenti.txt', 'w') as outfile:
    json.dump(response, outfile) 
        
  # apro e parso il file .json
  data = open('utenti.txt').read()
  data_parsed = json.loads(data)

  #creo e apro un file .csv per scrivere
  file_parsed = open('LindoMichoacan_10%Random_UsersData.csv', 'a')
    
  #creo il writer
  writer = csv.writer(file_parsed)

  #ciclo per estrarre dal file json i valori di id, età, genere ed etnia
  for person in data_parsed['user']:
     writer.writerow([person['id'],
                      person['age'],
                      person['gender'],
                      person['ethnicity']])
     
  
  #salvo e chiudo il file .csv
  file_parsed.close()

  file.SetContentFile('LindoMichoacan_10%Random_UsersData.csv') #The contents of the file
  file.Upload()

In [0]:
from clarifai.rest import ClarifaiApp
import time

def detect_clarifai(id, url_photo, data):
  app = ClarifaiApp(api_key='eced155788924ed9902063a28299b142')
  model = app.models.get(model_id="c0c0ac362b03416da06ab3fa36fb58e3")
  response = model.predict_by_url(url = '%s' % url_photo)
  
  if response['outputs'][0]['data'] != {}:
    x = 0
    while (x < len(response['outputs'][0]['data']['regions'])):
      age = response['outputs'][0]['data']['regions'][x]['data']['concepts'][0]['name']
      gender = response['outputs'][0]['data']['regions'][x]['data']['concepts'][20]['name']
      ethnicity = response['outputs'][0]['data']['regions'][x]['data']['concepts'][22]['name']

      data['user'].append({
          'id' : '%s' % id,
          'age' : '%s' % age,
          'gender' : '%s' % gender,
          'ethnicity' : '%s' % ethnicity,
      })
      x = x + 1
  else:
    return

In [0]:
#Da cambiare:
#id_csv_clarifai = link condivisibile del csv (clarifai) già fatto e il nome
#file_list = cartella da dove prendi le foto

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials
from google.colab import auth
from clarifai.rest import ApiError
import json
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# DOWNLOAD my_users.csv FROM DRIVE
id_csv_clarifai = '1fSqJYcczjUsbryfI7ekeqANaz-xD4PFT'  # FILE ID, got on google drive with condivision link
download = drive.CreateFile({'id': id_csv_clarifai})
download.GetContentFile('LindoMichoacan_10%Random_UsersData.csv')

#leggo il csv
csv_clarifai = pd.read_csv('LindoMichoacan_10%Random_UsersData.csv')

#Auto-iterate through all files that matches this query
file_list = drive.ListFile({'q': "'1eLi6-kfbLz_YJbUIGNLFq-RhtqvYLgyt' in parents and trashed=false"}).GetList()    #cartella da cui prendo le foto

data = {}
data['user'] = []
i=1
for file1 in file_list:
  id = file1['title'][0:22] 
  print(id,i)
  i = i +1
  df1 = csv_clarifai[csv_clarifai['id'].str.contains(id)]
  data = {}
  data['user'] = []
  if df1.empty :
    detect_clarifai(id, file1['thumbnailLink'], data)
    convert(data, download)


6m18Kd78MTNcT_O-dI8ZHA 0
AQ5R3cUUBiRkYGlgmY1nDw 1
QIPy_maubhs2y1-taLT7ZQ 2
l6smLfesX5zSdtGZ7PZFIw 3
F-lUKq7XhUNpoCeMnZRPGg 4
Z6QUufgf5KEVxGYrdKlHNw 5
JlTEv3ZAHTCDvt94fOjWOQ 6
h8okk52NQKV5CG0tO-sWWQ 7
chZwdGspJz5_xmm9dQnBkQ 8
oXcXtUjQvM5-ru-MSK3nlQ 9
dO9V1_-8a2xg5r0FdTsU4w 10
mcJ-A0OGmiWZGBbdWPMfrw 11
osBIW8l0husXz54j37_6bg 12
FKd5OWQnTa7V8TMIfqAQSA 13
F7y0PgmGN1P3MRGgjU_UNg 14
1cu_7a5qiN23UrAMNU7XTA 15
uIOcp-zn26Xp8cc3Kp2upg 16
57-vVLjMidkTYc9a39Au1g 17
TPRnMaeeiPjEKMb8U84wbA 18
N9hqTPQu2bmI8c2Pj8F1ww 19
qTOPqeg6jkBHqpcLG3JpSw 20
OBgUsBhI1od1Wt5tcxiCVQ 21
YmsYcbbjQk2l8pi7Xc-GbQ 22
7YoRbJXcMaA0FgpPpvV3Ig 23
jXa84PRtBZ50XD0a77YovA 24
ZRiVLbtEk9y5N8mzxav6fg 25
b2_JXAPP6HdO2Jl0MerMwQ 26
dhfgSH5_5KGhbNsG4t1l0Q 27
3G2_gN5571daBnQrIRAGBA 28
TWQ4bxTnfz40cZMFCE2W-g 29
VeaTf4qWSTiUeD8Iut1xzQ 30
w4VFONwd3DbYIY6dK_5n-w 31
uranKsfhvzJl1u5aVLao5A 32
b-46h9GeYHXEjCq1DWIP4Q 33
MX8_X0-CGOeu_Os8V8_VIQ 34
4F0wHkgYn0SvssiTrwDgCg 35
7juqeDI6dUe5R7fzRA4wKA 36
aAAnbXzummptZ-WlDa0YXA 37
bTxaQG8-RWXLetBnNt5afw